In [1]:
import pandas as pd
import numpy as np
import os
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# Function to clean NaN and infinite values
def clean_data(X_train, X_valid, X_test):
    X_train.fillna(X_train.mean(), inplace=True)
    X_valid.fillna(X_valid.mean(), inplace=True)
    X_test.fillna(X_test.mean(), inplace=True)
    return X_train, X_valid, X_test


# Load data
data_train = pd.read_pickle("clean_data/train/all_data_train.pkl")
data_valid = pd.read_pickle("clean_data/valid/all_data_valid.pkl")
data_test = pd.read_pickle("clean_data/test/all_data_test.pkl")

# Split the data into features (X) and target (y)
X_train = data_train.drop("class", axis=1)
X_valid = data_valid.drop("class", axis=1)
X_test = data_test.drop("class", axis=1)

y_train = data_train["class"]
y_valid = data_valid["class"]
y_test = data_test["class"]

# Encode the target variable (multi-class encoding)
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
encoded_y_valid = encoder.transform(y_valid)
encoded_y_test = encoder.transform(y_test)

# Clean the data (replace NaNs and infinities)
X_train, X_valid, X_test = clean_data(X_train, X_valid, X_test)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


# Function to build a GRU model with variable layers and neurons
def build_gru_model(
    num_neurons=32, num_layers=1, dropout_rate=0.2, input_shape=(None,)
):
    model = Sequential()
    for i in range(num_layers):
        if i == 0:
            model.add(
                GRU(
                    num_neurons,
                    return_sequences=(num_layers > 1),
                    input_shape=input_shape,
                )
            )
        else:
            model.add(GRU(num_neurons, return_sequences=(i != num_layers - 1)))
        model.add(Dropout(dropout_rate))

    # Output layer for classification
    model.add(Dense(4, activation="softmax"))

    # Compile the model
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return model


# Function to perform manual grid search
def manual_grid_search(X_train, y_train, X_valid, y_valid, param_grid):
    best_accuracy = 0
    best_model = None
    best_params = {}

    for num_neurons in param_grid["num_neurons"]:
        for num_layers in param_grid["num_layers"]:
            for dropout_rate in param_grid["dropout_rate"]:
                for batch_size in param_grid["batch_size"]:
                    print(
                        f"Training model with {num_neurons} neurons, {num_layers} layers, {dropout_rate} dropout, and {batch_size} batch size"
                    )

                    # Build and train the model
                    input_shape = (
                        X_train.shape[1],
                    )  # Adjust input shape to match your data
                    model = build_gru_model(
                        num_neurons, num_layers, dropout_rate, input_shape=input_shape
                    )
                    model.fit(
                        X_train,
                        y_train,
                        epochs=30,
                        batch_size=batch_size,
                        validation_data=(X_valid, y_valid),
                        verbose=1,
                    )

                    # Evaluate the model on validation data
                    _, accuracy = model.evaluate(X_valid, y_valid)
                    print(f"Validation accuracy: {accuracy}")

                    # Keep track of the best model
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_model = model
                        best_params = {
                            "num_neurons": num_neurons,
                            "num_layers": num_layers,
                            "dropout_rate": dropout_rate,
                            "batch_size": batch_size,
                        }

    return best_model, best_accuracy, best_params


# Define the hyperparameter grid manually
param_grid = {
    "num_neurons": [32, 64, 128],
    "num_layers": [1, 2],
    "dropout_rate": [0.1, 0.2, 0.3],
    "batch_size": [32, 64],
}

# Perform the manual grid search
best_model, best_accuracy, best_params = manual_grid_search(
    X_train, encoded_y_train, X_valid, encoded_y_valid, param_grid
)

# Display the best result
print(f"Best validation accuracy: {best_accuracy}")
print(f"Best parameters: {best_params}")

# Evaluate the best model on the test data
X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
_, test_accuracy = best_model.evaluate(X_test_reshaped, encoded_y_test)
print(f"Test accuracy: {test_accuracy}")

# Predict on the test data
y_pred = np.argmax(best_model.predict(X_test_reshaped), axis=1)

# Generate confusion matrix
cm = confusion_matrix(encoded_y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="g")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


# Function to combine pickles with class (existing function)
def combine_pickles_with_class(path, class_name):
    acc_files = []
    gyro_files = []

    # List all files in the directory
    for file in os.listdir(path):
        if file.startswith(class_name) and file.endswith("_acc.pkl"):
            acc_files.append(file)
        elif file.startswith(class_name) and file.endswith("_gyro.pkl"):
            gyro_files.append(file)

    combined_data_list = []

    # Process and combine acc and gyro files
    for i in range(len(acc_files)):
        acc_df = pd.read_pickle(os.path.join(path, acc_files[i]))
        gyro_df = pd.read_pickle(os.path.join(path, gyro_files[i]))

        # Drop the first column if there are 4 columns
        if len(acc_df.columns) == 4:
            acc_df.drop(acc_df.columns[0], axis=1, inplace=True)
            gyro_df.drop(gyro_df.columns[0], axis=1, inplace=True)

        acc_df.columns = ["ax", "ay", "az"]
        gyro_df.columns = ["gx", "gy", "gz"]

        # Combine the acc and gyro dataframes
        combined_df = pd.concat([acc_df, gyro_df], axis=1)

        combined_data_list.append(combined_df)

    # Concatenate all dataframes
    final_df = pd.concat(combined_data_list, ignore_index=True)
    return final_df


# Use the best model to predict on new data
new_data = combine_pickles_with_class("newdata/", "data")
new_data_scaled = scaler.transform(new_data)
new_data_reshaped = new_data_scaled.reshape(
    new_data_scaled.shape[0], 1, new_data_scaled.shape[1]
)
new_pred = np.argmax(best_model.predict(new_data_reshaped), axis=1)


# Function to plot sensor data with predicted classes
def plot_sensor_data(new_data, predicted_classes):
    fig, ax = plt.subplots(2, 1, figsize=(12, 10))

    # Plot accelerometer data
    ax[0].plot(new_data.index, new_data["ax"], label="ax", color="b", linewidth=1.5)
    ax[0].plot(new_data.index, new_data["ay"], label="ay", color="g", linewidth=1.5)
    ax[0].plot(new_data.index, new_data["az"], label="az", color="r", linewidth=1.5)
    ax[0].set_title("Accelerometer Data")
    ax[0].legend(loc="upper right")

    # Plot gyroscope data
    ax[1].plot(new_data.index, new_data["gx"], label="gx", color="b", linewidth=1.5)
    ax[1].plot(new_data.index, new_data["gy"], label="gy", color="g", linewidth=1.5)
    ax[1].plot(new_data.index, new_data["gz"], label="gz", color="r", linewidth=1.5)
    ax[1].set_title("Gyroscope Data")
    ax[1].legend(loc="upper right")

    # Adding vertical lines at points of class change
    class_changes = [
        i
        for i in range(1, len(predicted_classes))
        if predicted_classes[i - 1] != predicted_classes[i]
    ]
    for change_index in class_changes:
        ax[0].axvline(
            x=new_data.index[change_index], color="k", linestyle="--", alpha=0.5
        )
        ax[1].axvline(
            x=new_data.index[change_index], color="k", linestyle="--", alpha=0.5
        )

    plt.tight_layout()
    plt.show()


# Plot the new data and predicted classes
plot_sensor_data(new_data, new_pred)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


/var/folders/8z/bf26sjgx09qcs_63cnr3hp880000gn/T/ipykernel_35102/2150697309.py:85: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=build_gru_model, input_shape=(X_train.shape[1],))
2024-10-17 12:42:37.540973: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 12:42:37.541622: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild Ten

ValueError: Input 0 of layer "gru" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 6)